In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
pd.set_option('display.max_columns', None)

## 서울시 소방서 & 안전센터 위치정보

#### csv 파일 불러오기 -> 이미 전처리 된 파일

In [2]:
df_firesafe = pd.read_csv('firesafe.csv', encoding='UTF-8')

In [3]:
df_firesafe = df_firesafe.drop(df_firesafe.columns[0], axis=1)

In [4]:
df_firesafe.rename(columns={'주소':'자치구'}, inplace=True)
df_firesafe['flag']='소방서'

In [5]:
df_firesafe.head(3)

,이름,위도,경도,자치구,flag
0,영등포소방서,37.520641,126.913924,영등포구,소방서
1,동대문소방서,37.583794,127.050688,동대문구,소방서
2,용산소방서,37.531106,126.980942,용산구,소방서


## 서울시 도로구간 위치 정보

#### csv 파일 불러오기

In [6]:
df_street = pd.read_csv('street.csv', encoding='UTF-8')

C:\Users\LEEJUNGEUN\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_street.columns

Index(['순서', '시군구코드', '도로구간일련번호', '도로명', '도로명코드', '영문도로명', '고시일자', '도로명심의일자',
       '도로제정권자', '광역도로구분코드', '도로위계기능구분', '도로구간종속구분', '기점', '종점', '도로폭', '도로길이',
       '기초간격', '국도/지방도번호', '부여사유', '부여일자', '이동사유코드', '이동사유', '이동일자', '분기시군구코드',
       '분기도로구간일련번호', '입력방법', '교차로수', '발급여부', '위도', '경도'],
      dtype='object')

#### 도로 폭 4m미만 데이터만 추출 & column drop

In [8]:
df_street = df_street[(df_street['도로폭']<=4)]

In [9]:
df_street = df_street[['도로명', '위도', '경도', '시군구코드']]

### 시군구코드&이름 csv 파일 불러오기

In [10]:
df_gu = pd.read_csv('sigungu.csv', encoding='UTF-8')

In [11]:
df_gu = df_gu[['시군구코드', '시군구명_한글']]

#### 파일 병합 및 drop

In [12]:
df_street = pd.merge(df_street, df_gu)
df_street = df_street.drop(df_street.columns[3], axis=1)
df_street.rename(columns={'시군구명_한글':'자치구', '도로명' : '이름'}, inplace=True)

In [13]:
df_street['flag']= '도로'

In [14]:
df_street.head(3)

,이름,위도,경도,자치구,flag
0,옥인6가길,37.581420,126.965176,종로구,도로
1,자하문로,37.586980,126.969095,종로구,도로
2,사직로1길,37.573648,126.962948,종로구,도로


## 서울시 건물 정보

#### shp 파일 불러오기

In [15]:
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['figure.figsize'] = (10,10)

In [16]:
build_file = "seoul.shp"
build = gpd.read_file(build_file, encoding='euc-kr')

In [17]:
build[:1]

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A29,A30,A31,A32,A33,A34,A35,A36,A37,A38,geometry
0,1975197915054566808200000000,1111018300104540003,1111018300,서울특별시 종로구 평창동,1,일반,454-3,10001,1,일반건축물,2,일반건축물대방,None,0,111104100504,18301,0,00040,00000,None,0,주건축물,0.0,0.0,163.34,0.0,0.0,11,벽돌구조,01000,단독주택,0.0,2,0,None,19750204,0,0.0,2019-09-05,"POLYGON ((126.9772557944249 37.61244331076725,..."


#### column drop/rename & filtering

In [18]:
build = build[['A23', 'A28', 'A30', 'A32', 'A34', 'A35', 'A3', 'geometry']]
#column명 수정
col_list = ['면적','외장재', '이름', '지상층수','허가일자','사용승인일자', '주소', 'geometry']
build.columns=col_list
build.head(3)

,면적,외장재,이름,지상층수,허가일자,사용승인일자,주소,geometry
0,0.00,벽돌구조,단독주택,2,None,19750204,서울특별시 종로구 평창동,"POLYGON ((126.9772557944249 37.61244331076725,..."
1,34.65,일반목구조,단독주택,1,None,None,서울특별시 종로구 충신동,"POLYGON ((127.0057196556986 37.57560994223294,..."
2,0.00,벽돌구조,단독주택,2,None,19751001,서울특별시 종로구 홍지동,"POLYGON ((126.9570926512983 37.60274359589213,..."


In [19]:
###### 건출물 허가일자 필터링 ######

#float 변환
build['허가일자']=build['허가일자'].astype(float)
build['사용승인일자']=build['사용승인일자'].astype(float)

#fillna하고, int로 변환
build['허가일자']=build['허가일자'].fillna(0).astype(int)
build['사용승인일자']=build['사용승인일자'].fillna(0).astype(int)

#0인 허가일자 값 사용승인일자로 채우기 
build['허가일자']=np.where(build['허가일자']==0, build['사용승인일자'], build['허가일자'])

#0인값 없애기
build= build[build['허가일자']!=0]

#int로 변환
build['면적']=build['면적'].astype(int)

build.head()

,면적,외장재,이름,지상층수,허가일자,사용승인일자,주소,geometry
0,0,벽돌구조,단독주택,2,19750204,19750204,서울특별시 종로구 평창동,"POLYGON ((126.9772557944249 37.61244331076725,..."
2,0,벽돌구조,단독주택,2,19751001,19751001,서울특별시 종로구 홍지동,"POLYGON ((126.9570926512983 37.60274359589213,..."
3,0,벽돌구조,제2종근린생활시설,2,19610901,19610901,서울특별시 종로구 누하동,"POLYGON ((126.9688026330083 37.58022714931604,..."
4,0,벽돌구조,단독주택,2,19831107,19840715,서울특별시 종로구 평창동,"POLYGON ((126.965533507656 37.61175292373296, ..."
5,250,철근콘크리트구조,제2종근린생활시설,5,19821224,19821224,서울특별시 종로구 동숭동,"POLYGON ((127.0026164983 37.58147749843972, 12..."


#### dataframe 생성

In [20]:
df_build = pd.DataFrame(build.drop(columns='geometry'))

#### 위도/경도 & 자치구 생성

In [21]:
mypoints = [build.centroid]

list_point = []
for pp in mypoints:
    list_point.append([pp.x, pp.y])
np_point = np.array(list_point)

In [22]:
df_build['경도'] = np_point[0][0]
df_build['위도'] = np_point[0][1]

df_build['자치구']=df_build['주소'].apply(lambda e: e.split(' ')[1])

In [23]:
df_build.head(3)

,면적,외장재,이름,지상층수,허가일자,사용승인일자,주소,경도,위도,자치구
0,0,벽돌구조,단독주택,2,19750204,19750204,서울특별시 종로구 평창동,126.977174,37.612459,종로구
2,0,벽돌구조,단독주택,2,19751001,19751001,서울특별시 종로구 홍지동,126.957007,37.602759,종로구
3,0,벽돌구조,제2종근린생활시설,2,19610901,19610901,서울특별시 종로구 누하동,126.968768,37.580249,종로구


#### 화재 취약 외장재

In [24]:
### 외장재 ###
## 샌드위치패널이 주로 사용되는 공장&창고
## 목조건축

df_out = df_build[(df_build['이름']=='공장시설')|(df_build['이름']=='창고시설')|(df_build['외장재']=='일반목구조')]

df_out['flag'] = '외장재'

df_out.head(3)

C:\Users\LEEJUNGEUN\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,면적,외장재,이름,지상층수,허가일자,사용승인일자,주소,경도,위도,자치구,flag
7,38,일반목구조,제2종근린생활시설,3,20070718,20080220,서울특별시 종로구 팔판동,126.980976,37.583510,종로구,외장재
32,49,일반목구조,제2종근린생활시설,3,20071130,20080509,서울특별시 종로구 팔판동,126.981111,37.583362,종로구,외장재
36,47,일반목구조,단독주택,1,20070904,20080529,서울특별시 종로구 계동,126.986999,37.582433,종로구,외장재


#### 스프링클러 미설치

In [25]:
### 스프링클러 ###
## 필수설치건물제외
## -> 문화집회 및 운동시설 / 영화상영관
## 거의 연면적 600m2 이상
## 다중이용시설물 -> 문화/집회/종교/판매/운수/의료/숙박
## 단독주택 & 공동주택 6층 이상 건물
## --> 면적 50m2 이하인 6층 이하 소규모 단독주택이 주로 스프링쿨러 설치가 안되어있을 것으로 예측

df_sprinkler = df_build[df_build['면적']<600]

use_list = ['단독주택', '숙박시설', '동.식물관련시설']

df_sprinkler = df_sprinkler[df_sprinkler['이름'].isin(use_list)]

df_sprinkler = df_sprinkler[(df_sprinkler['이름']=='단독주택')&(df_sprinkler['지상층수']<6)&(df_sprinkler['면적']<50)]

df_sprinkler['flag'] = '스프링클러'

df_sprinkler.head(3)

,면적,외장재,이름,지상층수,허가일자,사용승인일자,주소,경도,위도,자치구,flag
0,0,벽돌구조,단독주택,2,19750204,19750204,서울특별시 종로구 평창동,126.977174,37.612459,종로구,스프링클러
2,0,벽돌구조,단독주택,2,19751001,19751001,서울특별시 종로구 홍지동,126.957007,37.602759,종로구,스프링클러
4,0,벽돌구조,단독주택,2,19831107,19840715,서울특별시 종로구 평창동,126.965434,37.611746,종로구,스프링클러


#### 노후 건물

In [26]:
### 노후 건물 ###
## ~ 1979년 모든 건물 (40년 이상 된 건물)

df_old = df_build[(df_build['허가일자']<19781231)]

df_old['flag'] = '노후'

df_old.head(3)

C:\Users\LEEJUNGEUN\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,면적,외장재,이름,지상층수,허가일자,사용승인일자,주소,경도,위도,자치구,flag
0,0,벽돌구조,단독주택,2,19750204,19750204,서울특별시 종로구 평창동,126.977174,37.612459,종로구,노후
2,0,벽돌구조,단독주택,2,19751001,19751001,서울특별시 종로구 홍지동,126.957007,37.602759,종로구,노후
3,0,벽돌구조,제2종근린생활시설,2,19610901,19610901,서울특별시 종로구 누하동,126.968768,37.580249,종로구,노후


#### 화재 취약지역

In [34]:
df_weak = pd.concat([df_out, df_sprinkler, df_old])
df_weak = df_weak[['이름', '위도', '경도', '자치구', 'flag']]

In [36]:
df_weak.head(3)

,이름,위도,경도,자치구,flag
7,제2종근린생활시설,37.583510,126.980976,종로구,외장재
32,제2종근린생활시설,37.583362,126.981111,종로구,외장재
36,단독주택,37.582433,126.986999,종로구,외장재


### 최종 파일

In [37]:
df_area = pd.concat([df_firesafe, df_street, df_weak])

In [38]:
df_area.head(3)

,이름,위도,경도,자치구,flag
0,영등포소방서,37.520641,126.913924,영등포구,소방서
1,동대문소방서,37.583794,127.050688,동대문구,소방서
2,용산소방서,37.531106,126.980942,용산구,소방서


In [48]:
df_area.to_csv('df_area.csv', encoding='ms949', header=True, index=False)

In [52]:
df_area.dtypes

이름       object
위도      float64
경도      float64
자치구      object
flag     object
dtype: object